In [1]:
%%capture
!pip3 install transformers
!pip3 install nltk emoji==0.6.0
!pip3 install umap-learn
!pip3 install pygsp

In [2]:
import sys, pickle, ast
import numpy as np
import pandas as pd
import torch
# import umap
from datetime import datetime
from tqdm import tqdm
# from transformers import AutoModel, AutoTokenizer
import matplotlib.pyplot as plt
import networkx as nx

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
FOLDER_PATH = '/content/drive/MyDrive/Knowledge Graphs 2022/'
%cd $FOLDER_PATH

/content/drive/.shortcut-targets-by-id/18wZgIc4f2VuuQ34uv_vRrBAOHvMWSEaq/Knowledge Graphs 2022


In [ ]:
!git clone https://github.com/VinAIResearch/BERTweet
sys.path.append('BERTweet/')
from BERTweet.TweetNormalizer import normalizeTweet

fatal: destination path 'BERTweet' already exists and is not an empty directory.


In [ ]:
!git clone https://github.com/ojus1/Date2Vec
sys.path.append('Date2Vec/')
from Model import Date2VecConvert

fatal: destination path 'Date2Vec' already exists and is not an empty directory.


In [5]:
all_tweets_df = pd.read_csv("data/all_tweets_50_topics.csv")
all_tweets_df = all_tweets_df.fillna('')
all_tweets_df

,id,text,time,user_id,user_screen_name,cleaned_text,topic,topic_probability
0,1420959115879145474,https://t.co/3bM9PoSxS6 #acoustic #orchestra #...,Fri Jul 30 04:07:08 +0000 2021,1404563992907976706,teknoaxe,,-1,0.000000
1,1420589578541428739,@DekeSlade Hey Rj! A link to my youtube chann...,Thu Jul 29 03:38:43 +0000 2021,1404563992907976706,teknoaxe,Hey Rj! A link to my youtube channel or the v...,-1,0.000000
2,1420533067400105985,@kraftoki @MinxMore How do you know?,Wed Jul 28 23:54:10 +0000 2021,1404563992907976706,teknoaxe,How do you know?,-1,0.845134
3,1420373718828679174,"@MinxMore Also, consider that ""immune for only...",Wed Jul 28 13:20:58 +0000 2021,1404563992907976706,teknoaxe,"Also, consider that immune for only a few mont...",-1,0.000000
4,1420373378653773829,@MinxMore It's like chicken pox....,Wed Jul 28 13:19:37 +0000 2021,1404563992907976706,teknoaxe,Its like chicken pox....,-1,0.000000
...,...,...,...,...,...,...,...,...
923380,1397974274367963136,@Baklava_in_CA @jordanbpeterson I got my pound.,Thu May 27 17:53:34 +0000 2021,218741203,mml111mml111,I got my pound.,-1,0.605857
923381,1397972983067693064,@nypost He tries to turn his hotel rooms into ...,Thu May 27 17:48:26 +0000 2021,218741203,mml111mml111,He tries to turn his hotel rooms into Vermont?,-1,0.000000
923382,1397972391708475392,@Baklava_in_CA @jordanbpeterson I've realized ...,Thu May 27 17:46:05 +0000 2021,218741203,mml111mml111,Ive realized we are never going to pay that de...,-1,0.000000
923383,1397813255523995648,"So, what do you think the long term side effec...",Thu May 27 07:13:44 +0000 2021,218741203,mml111mml111,"So, what do you think the long term side effec...",5,0.871033


In [7]:
# with open('bot detection/potential_bot_users.list', 'r') as f:
#   bot_user_list = [int(x) for x in f.read().split('\n')[:-1]]
# filtered_tweets_df = all_tweets_df[~all_tweets_df['user_id'].isin(bot_user_list)]
# filtered_tweets_df

### Tweet Embedding

1. document embedding

In [ ]:
bertweet = AutoModel.from_pretrained("vinai/bertweet-base").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tweet2normalized = dict()

for idx, row in tqdm(all_tweets_df.iterrows(), total=len(all_tweets_df)):
  tweet2normalized[row.id] = normalizeTweet(row.text)

100%|██████████| 923385/923385 [03:25<00:00, 4489.90it/s]


In [ ]:
test_sentence = list(tweet2normalized.values())[12199]

tokenized_sentence = tokenizer.encode(test_sentence, padding=True, truncation=True,max_length=128, add_special_tokens = True)

len(tokenized_sentence)

90

In [ ]:
tweet2tokens = dict()
for id, normalized_tweet in tqdm(tweet2normalized.items(), total=len(tweet2normalized)):
  tweet2tokens[id] = tokenizer.encode(normalized_tweet, padding=True, truncation=True,max_length=128, add_special_tokens = True)

100%|██████████| 923385/923385 [02:56<00:00, 5239.36it/s]


In [ ]:
len(list(tweet2tokens.values())[12199])

532

In [ ]:
batch_size = 10000
tweet2embedding = dict()

n_tweets_archived = 0
i = 0
print(f"resume from tweet #{n_tweets_archived}")
for id, tokens in tqdm(list(tweet2tokens.items())[n_tweets_archived:], total=len(tweet2tokens)-n_tweets_archived):

  input_ids = torch.tensor([tokens]).to("cuda")
  with torch.no_grad():
    features = bertweet(input_ids)
  tweet2embedding[id] = features.last_hidden_state[:, 0, :][0].to('cpu')

  if (i+1) % batch_size == 0:
    print(f"finished {i+1} tweets, storing...")
    with open("KG embedding/tweet2embedding.pickle", "wb") as f:
      pickle.dump(tweet2embedding, f, protocol=pickle.HIGHEST_PROTOCOL)
  i=i+1

with open("KG embedding/tweet2embedding.pickle", "wb") as f:
    pickle.dump(tweet2embedding, f, protocol=pickle.HIGHEST_PROTOCOL)
print(f"DONE")
with open("KG embedding/tweet2embedding.pickle", "rb") as f:
  test = pickle.load(f)

 16%|█▌        | 149155/923385 [17:51<1:32:34, 139.40it/s]

In [8]:
with open("KG embedding/tweet2embedding.pickle", "rb") as f:
  tweet2embedding = pickle.load(f)

In [9]:
X = torch.stack(list(tweet2embedding.values()))
tweet_ids = list(tweet2embedding.keys())
print(len(tweet_ids))
print(X.shape)

923385
torch.Size([923385, 768])


In [10]:
pca = PCA(n_components=236, whiten=True)
X_pca = pca.fit_transform(X)
print(X_pca.shape)

(923385, 236)


In [11]:
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))

[0.09852814 0.07148885 0.03782106 0.02536357 0.02324777 0.02228377
 0.01920785 0.01763199 0.01709846 0.01508476 0.014763   0.0145819
 0.01360044 0.01337704 0.01293045 0.01233775 0.01210587 0.01141704
 0.01074508 0.01022558 0.00981582 0.00966663 0.00928262 0.00913458
 0.00883717 0.0084956  0.00837816 0.00816728 0.00801296 0.0079084
 0.00769167 0.00749003 0.00741712 0.00721406 0.00696808 0.00671954
 0.00656326 0.00632929 0.00621854 0.00613725 0.00595517 0.00587776
 0.00570373 0.00550746 0.00530491 0.00522517 0.00515468 0.00501474
 0.0049178  0.00472176 0.00468577 0.0045526  0.0045456  0.00439136
 0.00434166 0.00427137 0.00419466 0.0041656  0.00399705 0.00393562
 0.00383986 0.00378701 0.00372887 0.00371073 0.00358418 0.00351139
 0.0035007  0.00346732 0.00343316 0.00338936 0.00330623 0.00326468
 0.00323915 0.00321337 0.0031655  0.00310718 0.00309086 0.0030067
 0.00290195 0.00288667 0.00281706 0.0027755  0.00270781 0.00265645
 0.00261682 0.00256007 0.00253683 0.00252579 0.00245677 0.0023941

In [12]:
tweet2embedding_pca = dict()
for idx, tweet_id in tqdm(enumerate(tweet_ids), total=len(tweet_ids)):
  tweet2embedding_pca[tweet_id] = X_pca[idx]

tweet2embedding_pca[list(tweet2embedding_pca.keys())[0]].shape

100%|██████████| 923385/923385 [00:00<00:00, 1079899.62it/s]


(236,)

In [13]:
with open("KG embedding/tweet2embedding_pca.pickle", "wb") as f:
  pickle.dump(tweet2embedding_pca, f, protocol=pickle.HIGHEST_PROTOCOL)

with open("KG embedding/tweet2embedding_pca.pickle", "rb") as f:
  test = pickle.load(f)

print(len(test) == len(tweet2embedding_pca))

True


3. timestamp as positional encoding

In [ ]:
all_tweets_df['time'] = pd.to_datetime(all_tweets_df['time'], infer_datetime_format=True)
all_tweets_df  = all_tweets_df.set_index('id')
tweet2time = all_tweets_df['time'].to_dict()

In [ ]:
NUM_SECONDS_IN_A_MIN = 60
start_time = datetime(2018, 1, 1, tzinfo=timezone.utc)

def convert_time(time):
  time_tuple = time.timetuple()
  # Date-Time format: 13:23:30 2019-7-23
  return torch.Tensor([time_tuple[3], time_tuple[4], time_tuple[5], time_tuple[0], time_tuple[1], time_tuple[2]]).float()


d2v = Date2VecConvert(model_path="Date2Vec/d2v_model/d2v_98291_17.169918439404636.pth")

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'Model.Date2Vec' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump

In [ ]:
tweet2timevec = dict()
for id, time in tqdm(tweet2time.items(), total=len(tweet2time)):
  tweet2timevec[id] = d2v(convert_time(time))

100%|██████████| 923385/923385 [01:20<00:00, 11494.56it/s]


In [ ]:
with open("KG embedding/tweet2timevec.pickle", "wb") as f:
  pickle.dump(tweet2timevec, f, protocol=pickle.HIGHEST_PROTOCOL)

with open("KG embedding/tweet2timevec.pickle", "rb") as f:
  test = pickle.load(f)

print(len(test) == len(tweet2timevec))

True


In [14]:
with open("KG embedding/tweet2timevec.pickle", "rb") as f:
  tweet2timevec = pickle.load(f)

In [15]:
scaler = StandardScaler()
timevec = torch.stack(list(tweet2timevec.values()))
print(timevec.shape)
normalized_timevec = scaler.fit_transform(timevec)
print(normalized_timevec.shape)

torch.Size([923385, 64])
(923385, 64)


In [16]:
tweet2timevec_norm = dict()
tweet_ids = list(tweet2timevec.keys())
for idx, tweet_id in tqdm(enumerate(tweet_ids), total=len(tweet_ids)):
  tweet2timevec_norm[tweet_id] = normalized_timevec[idx]

tweet2timevec_norm[list(tweet2timevec_norm.keys())[0]].shape

100%|██████████| 923385/923385 [00:00<00:00, 1109940.45it/s]


(64,)

concat features

In [17]:
print(tweet2timevec_norm[list(tweet2timevec.keys())[0]])
print(tweet2embedding_pca[list(tweet2embedding_pca.keys())[0]])

[-2.00704645  1.21680057  0.69184603 -1.22783007 -2.08100272 -1.57406539
 -2.07413419  0.0942709  -0.38960643 -0.15787364 -2.2556066   0.8356684
  1.50796423  0.11272071 -0.77094285 -1.79462235 -0.06999794 -0.57714757
 -0.24140124 -0.05000331 -1.61223192  1.83097518 -0.7556505  -1.6517176
  0.02699771  1.23344382  0.44401194  0.36818133  1.87810164  0.93273466
 -0.06911724  0.86448577 -1.27836736  1.0352459  -1.35597237 -0.62631623
 -0.93049579  0.72647741  0.83856546  0.59794701  0.18539551  1.26459768
 -0.08545183  1.14281724  0.99468523  0.84006407 -1.39017314 -1.39538819
 -1.18645856  1.25400178  0.65378406 -0.47254596 -0.05985119  1.22566134
 -0.9691339  -0.57476454  1.40502685 -1.06960708 -1.15059921  1.41438098
  0.85014088 -0.56852553  1.40826558 -1.34896162]
[ 0.19074525  0.30474671  0.23711104  0.19694799  0.27181587 -1.64385686
 -1.7205512   0.21311571  0.05907074  0.60015047 -1.38514646  0.00491863
 -2.16639017  0.38443457 -1.21044978  1.26796414 -0.34039666 -1.42842517
  1

In [18]:
tweet_ids = list(tweet2embedding_pca.keys())
tweet_embedding = dict()
for tweet_id in tqdm(tweet_ids, total=len(tweet_ids)):
  embedding = np.concatenate((tweet2embedding_pca[tweet_id], tweet2timevec_norm[tweet_id]))
  assert len(embedding) == 300
  tweet_embedding[tweet_id] = embedding

100%|██████████| 923385/923385 [00:04<00:00, 198357.45it/s]


In [19]:
tweet_embedding[list(tweet_embedding.keys())[0]].shape

(300,)

In [20]:
with open('KG embedding/tweet_embedding.pickle', 'wb') as f:
  pickle.dump(tweet_embedding, f, protocol=pickle.HIGHEST_PROTOCOL)

with open('KG embedding/tweet_embedding.pickle', 'rb') as f:
  test = pickle.load(f)

print(len(test) == len(tweet_embedding))

True


### User Embedding

In [ ]:
G = nx.read_gexf('social network/snapshots/graph/full.gexf')
# nx.write_weighted_edgelist(G, "social network/snapshots/graph edges/full.weighted.edgelist")

In [ ]:
%cd $FOLDER_PATH
!git clone https://github.com/phanein/deepwalk
%cd deepwalk
!pip3 install -r requirements.txt
!python setup.py install
!python setup.py build

/content/drive/.shortcut-targets-by-id/18wZgIc4f2VuuQ34uv_vRrBAOHvMWSEaq/Knowledge Graphs 2022
fatal: destination path 'deepwalk' already exists and is not an empty directory.
/content/drive/.shortcut-targets-by-id/18wZgIc4f2VuuQ34uv_vRrBAOHvMWSEaq/Knowledge Graphs 2022/deepwalk
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
running install
running bdist_egg
running egg_info
writing deepwalk.egg-info/PKG-INFO
writing dependency_links to deepwalk.egg-info/dependency_links.txt
writing entry points to deepwalk.egg-info/entry_points.txt
writing requirements to deepwalk.egg-info/requires.txt
writing top-level names to deepwalk.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
adding license file 'AUTHORS.rst'
writing manifest file 'deepwalk.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creati

In [ ]:
# !deepwalk --format edgelist --input ../social\ network/snapshots/graph\ edges/full.edgelist --max-memory-data-size 0 --number-walks 80 --representation-size 252 --walk-length 40 --window-size 10 \
# --workers 2 --output ../KG\ embedding/social_network.embeddings

Number of nodes: 26041
Number of walks: 2083280
Data size (walks*length): 83331200
Data size 83331200 is larger than limit (max-memory-data-size: 0).  Dumping walks to disk.
Walking...
Counting vertex frequency...
Training...


In [ ]:
!deepwalk --format edgelist --input ../social\ network/snapshots/graph\ edges/full.edgelist --max-memory-data-size 0 --number-walks 80 --representation-size 296 --walk-length 40 --window-size 10 \
--workers 2 --output ../KG\ embedding/social_network.embeddings

Number of nodes: 26041
Number of walks: 2083280
Data size (walks*length): 83331200
Data size 83331200 is larger than limit (max-memory-data-size: 0).  Dumping walks to disk.
Walking...
Counting vertex frequency...
Training...


In [ ]:
%cd $FOLDER_PATH

/content/drive/.shortcut-targets-by-id/18wZgIc4f2VuuQ34uv_vRrBAOHvMWSEaq/Knowledge Graphs 2022


In [ ]:
user_embedding_df = pd.read_csv('KG embedding/archive/social_network.embeddings', skiprows=1, header=None, sep=' ')
user_embedding_df

,0,1,2,3,4,5,6,7,8,9,...,243,244,245,246,247,248,249,250,251,252
0,25073877,-0.015305,0.024044,0.110244,0.004643,0.073365,0.020981,-0.018111,0.001397,-0.149274,...,0.053907,-0.190131,0.072977,-0.015920,-0.075364,-0.106610,-0.088970,0.072904,0.076520,-0.216593
1,1349149096909668363,-0.035061,0.025451,0.039147,-0.012328,0.001933,0.301734,-0.099378,-0.000545,-0.088371,...,0.036099,-0.066456,0.057151,0.079173,0.034050,0.014647,0.081677,-0.025059,0.002302,0.017575
2,10228272,-0.123200,0.020456,0.229902,-0.053655,0.067059,0.030361,0.032699,-0.007746,0.007544,...,0.027586,0.077509,0.006645,0.102934,-0.038339,-0.145000,-0.108141,0.161255,-0.071073,-0.037213
3,939091,-0.016069,0.118787,0.230988,0.114549,-0.029909,0.036747,0.087223,-0.000885,-0.089803,...,-0.089371,-0.019128,0.144553,-0.054249,-0.043045,-0.006374,-0.106355,-0.026493,0.080084,-0.137427
4,592730371,0.011170,-0.065804,0.021136,0.010625,0.151987,-0.014716,0.066956,-0.123882,-0.000512,...,-0.108169,0.070422,0.154605,0.038084,-0.207455,0.034915,0.004336,0.121383,-0.008061,-0.033066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26036,3293505444,-0.026143,0.067688,-0.040391,-0.394679,0.025742,0.050757,-0.123877,-0.112143,-0.457814,...,-0.207087,-0.212286,-0.297584,0.169915,-0.032383,-0.072643,0.325037,-0.323479,-0.207752,-0.047244
26037,954919661308559362,-0.014738,0.262085,0.272612,-0.101640,0.179012,0.236179,-0.047122,-0.124601,0.100255,...,0.148641,0.623474,0.080798,-0.170408,0.144014,0.018745,-0.108878,-0.251369,-0.384466,-0.335112
26038,794450313797337088,-0.189663,-0.077866,0.219794,-0.240731,0.002690,0.268689,-0.521236,0.131472,-0.160147,...,-0.141431,-0.134919,0.214034,-0.159092,0.025477,0.458900,0.456361,0.215910,-0.070104,0.002804
26039,3185316326,-0.079612,-0.315948,-0.058771,0.165482,0.268402,0.072809,-0.204464,-0.515671,0.148860,...,0.061926,0.054417,-0.074539,0.029414,-0.551013,-0.027845,-0.056621,0.010328,-0.235589,0.111197


In [ ]:
user_embedding = dict()
for idx, row in tqdm(user_embedding_df.iterrows(), total=len(user_embedding_df)):
  user_embedding[row[0]] = user_embedding_df.iloc[idx, 1:].values

with open("KG embedding/user_embedding_deepwalk.pickle", "wb") as f:
  pickle.dump(user_embedding, f, protocol=pickle.HIGHEST_PROTOCOL)

with open("KG embedding/user_embedding_deepwalk.pickle", "rb") as f:
  test = pickle.load(f)

print(len(test) == len(user_embedding))

True


In [ ]:
# with open('data/big_dict.pickle', 'rb') as f:
#   all_tweets_info = pickle.load(f)

# all_tweets_info_df = pd.DataFrame.from_dict(all_tweets_info)
# all_tweets_info_df['user_id'] = all_tweets_info_df.user.apply(lambda x: x['id'])
# user_meta_info_df = all_tweets_info_df[['user_id', 'favourites_count', 'followers_count', 'friends_count', 'statuses_count']].drop_duplicates().set_index('user_id')

In [ ]:
# user_meta_info_df.to_csv("KG embedding/user_meta_info.csv")
user_meta_info_df = pd.read_csv("KG embedding/user_meta_info.csv")
user_meta_info_df

In [ ]:
user_embedding_df = pd.read_csv('KG embedding/social_network.embeddings', skiprows=1, header=None, sep=' ')
user_embedding_df

In [ ]:
user_meta_info_df[user_meta_info_df['user_id'] == 1404563992907976706].iloc[:, 1:].values[0]

In [ ]:
raw_embedding = dict()
whole_user_list = user_embedding_df[0].values
user_list = user_meta_info_df['user_id'].values
meta_info = []
for idx, user in tqdm(enumerate(whole_user_list), total=len(whole_user_list)):
  raw_embedding[user] = user_embedding_df.iloc[idx, 1:].values
  if user in user_list:
    meta_info.append(user_meta_info_df[user_meta_info_df['user_id'] == user].iloc[:, 1:].values[0])

np.stack(meta_info).shape

In [ ]:
meta_info = np.stack(meta_info)
max_values = np.max(meta_info, axis=0)
user_embedding = dict()

for idx, user in tqdm(enumerate(whole_user_list), total=len(whole_user_list)):
  user_embedding[user] = user_embedding_df.iloc[idx, 1:].values
  if user in user_list:
    user_embedding[user] = np.concatenate((user_embedding[user], np.divide(user_meta_info_df[user_meta_info_df['user_id'] == user].iloc[:, 1:].values[0], max_values)))
  else:
    user_embedding[user] = np.concatenate((user_embedding[user], np.zeros(4)))
  assert len(user_embedding[user]) == 300

In [ ]:
with open('KG embedding/user_embedding.pickle', 'wb') as f:
  pickle.dump(user_embedding, f, protocol=pickle.HIGHEST_PROTOCOL)

with open('KG embedding/user_embedding.pickle', 'rb') as f:
  test = pickle.load(f)

print(len(test) == len(user_embedding))